<p style="font-family: Arial; font-size:3.70em;color:purple; font-style:bold">
Creating Sankey Using Plotly 
</p>
<br>


<p>Here, I am going to build sankey diagram/sankey plot using plotly.</p>
<p>There are few points to keep in mind while creating Sankey using plotly.</p>

1. plotly's sankey representation accept values of source and target as an **integers** not **strings.**
2. So, we have to come up with representation to describe our source and target with unique values associated with them. In other words, we have to prepare some kind of mapping of our source and target, which can be mapped back to our dataframe and which in turn can be used as source and target value.
3. Labels of nodes in plotly must contains source and target both labels. (which is same in any other library or framework too, I guess.)
4. Remember, source value starts with 0 and label which is very first in label list will be assigned to this source. Similarly, second lable assigned to source value 1 and so on and so forth. This is general convention. I found it during exploring examples and other resources on plotly sankey.
<br><br>
<b>Even if you don't fully understands above points. I am going to elaborate them while doing Data Transformation.</b>

In [1]:
import plotly
import pandas as pd
import numpy as np
from plotly.graph_objs import Sankey

In [2]:
data = pd.read_csv('FDU_Data.csv', sep =",")
print(type(data))
print("Shape will show number of rows and columns in DataFrame (rows,columns):",data.shape)
data.head()

<class 'pandas.core.frame.DataFrame'>
Shape will show number of rows and columns in DataFrame (rows,columns): (1262, 43)


,REC_ID,FILETERM,FTERMA,FILEDATE,COURSE,CRSLEVEL,CDEPT,CNUM,CTITLE,MINCREDS,...,SATVERB,SATMATH,SATWRITE,ACT,Wordcount,ZYBooks_Timeliness,ZYBooks_Curiosity,Weighted_Midterm_Total,Weighted_Final_Total,Word_Count
0,3878,201509,201509,9/1/2015,EGTM_2232,2,EGTM,2232,Mechanical Measurement/Devices,3,...,450,460,480,,,1500.0,700.0,68.6,90.25,296
1,3878,201509,201509,9/1/2015,ENGR_1301,1,ENGR,1301,"Engr Pract, Graphics and Des",3,...,450,460,480,,,1500.0,700.0,68.6,90.25,296
2,3878,201509,201509,9/1/2015,ENWR_1101,1,ENWR,1101,Academic Writing,3,...,450,460,480,,,1500.0,700.0,68.6,90.25,296
3,3878,201509,201509,9/1/2015,MATH_1105,1,MATH,1105,College Algebra,4,...,450,460,480,,,1500.0,700.0,68.6,90.25,296
4,3878,201602,201602,2/1/2016,EGTC_1223,1,EGTC,1223,Introduction to CAD,2,...,450,460,480,,,1500.0,700.0,68.6,90.25,296


<p>We are going to need course and grade columns only to generate sankey plot. In which, <b>Course Title</b> will be our <b>Source</b>, <b>Grade</b> will be our <b>Target</b> and <b>Value</b> will be number of students who got A, A-, B+.. grades in the Source Subject.</p>

In [3]:
df = data[['REC_ID','CTITLE','Grade']]
print(df.shape)
df[:3]

(1262, 3)


,REC_ID,CTITLE,Grade
0,3878,Mechanical Measurement/Devices,C
1,3878,"Engr Pract, Graphics and Des",B+
2,3878,Academic Writing,B


## Performing Data Cleaning operations as I did in first Jupyter Notebook.

In [4]:
df.isnull().any()

REC_ID    False
CTITLE    False
Grade     False
dtype: bool

In [5]:
df.dtypes

REC_ID     int64
CTITLE    object
Grade     object
dtype: object

In [6]:
df['CTITLE'].str.contains(' ').any()

True

In [7]:
df['Grade'].str.contains(' ').any()

True

In [8]:
df = df[['CTITLE','Grade']].replace(' ',np.nan)

In [9]:
df = df.dropna()

In [10]:
df.shape

(1187, 2)

### We will need unique course title and grade list to create mapping for our final data frame.

In [11]:
Ctitle = df['CTITLE'].unique().tolist()
print(len(Ctitle))
Ctitle.sort()
print(Ctitle[:10])

166
['APreview of Profssionl Nursng', 'Academic Research and Writing', 'Academic Writing', 'Adv. Engineering Programming', 'Advanced Database', 'Advanced Database Systems', 'Advanced Engineering Math', 'Algebra Skills', 'Applied Statistics I', 'Applied Thermodynamics']


In [12]:
grade = df['Grade'].unique().tolist()
del grade[-3:]
grade.sort()
print(grade)
print(len(grade))

['A', 'A-', 'B', 'B+', 'B-', 'C', 'C+', 'C-', 'D', 'F']
10


## Pandas GroupBy abilities
<br>
<p>GroupBy performs the same operation as it does in Database. It is mainly used to perform three operation on data.</p>
* **Splitting** the data into groups based on some criteria
* **Applying** a function to each group independently
* **Combining** the results into a data structure
<br><p>For detailed information click the link below.</p>
<a href = "http://pandas.pydata.org/pandas-docs/stable/groupby.html?highlight=groupby">Pandas GroupBy</a>
<br><br>
<b style = "font-size: 16px">Here, groupby(['CTITLE','Grade']) will split the data into buckets by Course Title and then splitting the record inside each Course Title bucket by Grade.</b>

In [13]:
grp_by = df.groupby(['CTITLE','Grade'])

In [14]:
grp_by

## What and How to display?

<p>Here, We want to count number of rows in each grouping. For that <b>size()</b> is used on group by. So, you will see number of grades received in each course.</p>

### For more detailed information about our next two steps in group by click the link below.
<a href ="https://community.modeanalytics.com/python/tutorial/pandas-groupby-and-python-lambda-functions/">GroupBy size() and unstack() functions.</a>

In [15]:
grp_by.size()

CTITLE                          Grade
APreview of Profssionl Nursng   A-        1
Academic Research and Writing   A         5
                                A-        3
                                B         7
                                B+        4
                                B-        9
                                C         2
                                C+        3
                                D         3
                                F         2
Academic Writing                A         5
                                A-       11
                                B         6
                                B+        5
                                B-        7
                                C         3
                                C+        4
                                C-        3
                                D         4
                                F         5
Adv. Engineering Programming    A         1
                                A-    

## GroupBy unstack() will convert the result into more readable format.

In [16]:
count_grades = grp_by.size().unstack().reset_index()
count_grades[:3]

Grade,CTITLE,A,A-,B,B+,B-,C,C+,C-,D,F,I,NC,P
0,APreview of Profssionl Nursng,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Academic Research and Writing,5.0,3.0,7.0,4.0,9.0,2.0,3.0,NaN,3.0,2.0,NaN,NaN,NaN
2,Academic Writing,5.0,11.0,6.0,5.0,7.0,3.0,4.0,3.0,4.0,5.0,NaN,NaN,NaN


Removing unnecessary columns from count_grades.

In [17]:
del count_grades['NC']


In [18]:
del count_grades['P']


In [19]:
del count_grades['I']

<p>As we can observ that we have NaN values. However, we deal with NaN before too. The only difference is that before we removed the records from dataframe. But, over here we can not do that. It would be logically incorrect and would yield nothing. So, what we can do is <b>fill null values with 0 </b>which is more appropriate in our case.</p>
<br>
<p>There are two ways to do it.</p>

1. Using **replace(np.nan, 0, inplace = True)**
2. Or using **fillna(np.nan, 0, inplace = True)**

<p>They both are doing the same task.</p>

In [20]:
count_grades.replace(np.nan, 0, inplace = True)

In [21]:
count_grades.tail()

Grade,CTITLE,A,A-,B,B+,B-,C,C+,C-,D,F
161,Weight Training,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
162,World Dance Apprec & Practice,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
163,World History Since 1500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164,World History to 1500,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,Yoga,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


# Reshaping Data using Pandas melt
<br>
<p>Types of Table formats.</p>

* stacked format: The individual observations are stacked on top of each other.
* record format: Each row is a single record, i.e. a single observation.
* long format: This format will be long in the vertical direction as opposed to wide in the horizontal direction.
* wide format: Wider rather than longer.(For Example our count_grade DataFrame.)
* unstacked format: The individual observations do not stacked on top of each other.
<br>
<p>Our Table is in wide format. We have to unpivot the data frame which will be helpful, because it can reshape the data the way we want. </p>
<br>
<p>To refer to more resources for <b>melt.</b> The links are given below.</p>
<a href = "https://hackernoon.com/reshaping-data-in-python-fa27dda2ff77"> Reshaping Data in Python.</a>
<br>
<a href = "https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html"> pd.melt documentation.</a>

### What does melt() do?
<b>melt() function takes 3 arguments:</b>

1. First, the DataFrame to melt.
2. Second, ID variables to keep (Pandas will melt all of the other ones).
3. Finally, a name for the new, melted variable.

In [22]:
dfm = pd.melt(count_grades,id_vars = ['CTITLE'] # columns to keep
              , var_name = 'target', value_name = 'value')

In [23]:
dfm.columns = ['source','target','value']

In [24]:
print(dfm.size)
dfm[:5]

4980


,source,target,value
0,APreview of Profssionl Nursng,A,0.0
1,Academic Research and Writing,A,5.0
2,Academic Writing,A,5.0
3,Adv. Engineering Programming,A,1.0
4,Advanced Database,A,1.0


#### Filtering out values as we do not want stubjects having grades values 0.

In [25]:
filter_zero = dfm['value'] > 0 

In [26]:
dfm = dfm[filter_zero]
dfm.size

1419

In [27]:
dfm.reset_index(drop = True)[:5]

,source,target,value
0,Academic Research and Writing,A,5.0
1,Academic Writing,A,5.0
2,Adv. Engineering Programming,A,1.0
3,Advanced Database,A,1.0
4,Algebra Skills,A,3.0


In [28]:
dfm.isnull().any()

source    False
target    False
value     False
dtype: bool

In [29]:
dfm.duplicated().any()

False

## Filter out the subjects having less then 10 students to optimize our plotting

In [30]:
course_list = []

In [31]:
i = 0
while i < len(Ctitle):
    filt_sub = dfm['source'] == Ctitle[i]
    new_df = dfm[filt_sub]
    filt_stu = new_df['value'].values.sum()
    if filt_stu >= 10:
        course_list.append(Ctitle[i])
    i+=1

In [32]:
print(len(course_list))
course_list[:3]

32


['Academic Research and Writing', 'Academic Writing', 'Calculus I']

# Sankey Creating.

<p>This would be few step process.</p>

* Now that we have courses with more than 9 students, we can create the labels required for the nodes of Sankey. Remember as I said before that, this list should contain source and target both labels. 

In [33]:
lbls = course_list + grade
lbls[-4:]

['C+', 'C-', 'D', 'F']

* Creating Mapping for source values, as we have 32 courses with more than 9 students. We can map those 32 courses with range of first 32 numbers starting from 0. This mapping can be done as follows. 
    * Creating list of numbers to map.
    * Zipping the course_list and number list to create the dictionary. The reason behind to creating dictionary is that, we would replace courses with values associated with it. So, course name would be key and integers will be value for that course.

In [34]:
crs_zip = list(range(0,32))

In [35]:
src_mapping = dict(zip(course_list, crs_zip))
src_mapping

{'Academic Research and Writing': 0,
 'Academic Writing': 1,
 'Calculus I': 2,
 'Calculus II': 3,
 'Circuits I': 4,
 'Circuits II': 5,
 'College Algebra': 6,
 'Computer Programming I': 7,
 'Computer Programming II': 8,
 'ComputerTechnlgies&CyberCrime': 9,
 'Computers & Computer Software': 10,
 'Cross Cultural Perspectives': 11,
 'Digital System Design': 12,
 'Eng for Occupational Purposes': 13,
 'Engr Pract, Graphics and Des': 14,
 'General Chemistry I': 15,
 'General Physics I': 16,
 'General Physics II': 17,
 'Intro to Computer Science': 18,
 'Introduction to CAD': 19,
 'LAB: General Chemistry I': 20,
 'LAB: Physics I': 21,
 'LAB: Physics II': 22,
 'Mechanical Measurement/Devices': 23,
 'Mod Tech:Prin/Appl/Impacts': 24,
 'Precalculus': 25,
 'Prep for Professional Life': 26,
 'Program Lang in Engineering': 27,
 'Statics': 28,
 'Strength of Materials': 29,
 'Technical Communications': 30,
 'Transitioning to Univer Life': 31}

* Similarly, creating target mapping. We have grades from A to F which is mapped to 10 distinguish numbers. 

In [36]:
tr_zip = list(range(32,42))

In [37]:
trg_mapping = dict(zip(grade,tr_zip))
trg_mapping

{'A': 32,
 'A-': 33,
 'B': 34,
 'B+': 35,
 'B-': 36,
 'C': 37,
 'C+': 38,
 'C-': 39,
 'D': 40,
 'F': 41}

#### Creating color mapping for links

In [38]:
clr_zip = ['#31a354','#a1d99b','#FFFF32','#FFFF66','#CCCC00','#FFAE19','#FFC966','#CC8400','#e34a33','#f03b20']

In [39]:
link_clr_mapping = dict(zip(grade,clr_zip))
link_clr_mapping

{'A': '#31a354',
 'A-': '#a1d99b',
 'B': '#FFFF32',
 'B+': '#FFFF66',
 'B-': '#CCCC00',
 'C': '#FFAE19',
 'C+': '#FFC966',
 'C-': '#CC8400',
 'D': '#e34a33',
 'F': '#f03b20'}

## Time to apply filters

1. Firstly, I will create data frame copy.
2. Secondly, Filter out the courses less then 10 students using isin() method. It checks if the values in particular column or whole dataframe present in given list or not.
3. Applying Source and Target Mapping.

In [40]:
dfm_cpy = dfm.copy()
print(dfm_cpy.size)
dfm_cpy[:3]

1419


,source,target,value
1,Academic Research and Writing,A,5.0
2,Academic Writing,A,5.0
3,Adv. Engineering Programming,A,1.0


In [41]:
dfm_cpy = dfm_cpy[dfm_cpy['source'].isin(course_list)]

In [42]:
print(dfm_cpy.size)
dfm_cpy[:5]

648


,source,target,value
1,Academic Research and Writing,A,5.0
2,Academic Writing,A,5.0
15,Calculus I,A,4.0
16,Calculus II,A,3.0
19,Circuits I,A,2.0


### To map link color for each grades we need to have grades values for each record. For that we need to preserve our target column values.

In [43]:
link_color = dfm_cpy['target'].copy()
link_color[:4]

1     A
2     A
15    A
16    A
Name: target, dtype: object

In [44]:
dfm_cpy.replace(src_mapping, inplace = True)

In [45]:
dfm_cpy.replace(trg_mapping, inplace = True)

### You would notice that each course and grade is replaced by values assigned to them.

In [46]:
dfm_cpy[:5]

,source,target,value
1,0,32,5.0
2,1,32,5.0
15,2,32,4.0
16,3,32,3.0
19,4,32,2.0


### Creating new column to map target colors. 

In [47]:
dfm_cpy['link_color'] = link_color

In [48]:
dfm_cpy[:2]

,source,target,value,link_color
1,0,32,5.0,A
2,1,32,5.0,A


### Mapping colors to target links

In [49]:
dfm_cpy.replace(link_clr_mapping, inplace = True)

In [50]:
dfm_cpy[:4]

,source,target,value,link_color
1,0,32,5.0,#31a354
2,1,32,5.0,#31a354
15,2,32,4.0,#31a354
16,3,32,3.0,#31a354


## Using plotly offline to generate diagram on localhost.

In [51]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

## Here, notice that data_trace is dictionary, label = lbls is lables for nodes, source, target and value are assigned the data frame we created.

In [52]:
data_trace = dict(
    type='sankey',
    domain = dict(
      x =  [0,1],
      y =  [0,1]
    ),
    orientation = "h",
    visible = True,
    valueformat = ".0f",
    valuesuffix = " Students",
    node = dict(
      pad = 15,
      thickness = 30,
      line = dict(
        color = "black",
        width = 0.5
      ),
      label =  lbls,
      #color = dfm_cpy['node_color']
    ),
    link = dict(
      source = dfm_cpy['source'],
      target = dfm_cpy['target'],
      value = dfm_cpy['value'],
      color = dfm_cpy['link_color'],# Here we can add colors for each link which connected source to target.
  )
)

layout =  dict(
    title = "Course Sankey Diagram",
    height = 1000,
    width = 1300,
    font = dict(
      size = 12
    ),    
)


In [53]:
fig = Figure(data=[data_trace], layout=layout)
plotly.offline.plot(fig, validate=False)

'file://C:\\Users\\Hemang\\juPyterNotebook_Practice\\Student_Data\\temp-plot.html'

### Ipysankey Widget has also capability to represent sankey diagram. However, I noticed that big visualization should be generated separately because, Jupyter Notebook could run into problem while generating inline plots.

#### Below, you will see small example of sankey diagram for one course having 38 students. 

In [54]:
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout

In [55]:
layout = Layout(width="900", height="900")

In [56]:
filt_subject = dfm['source'] == 'Academic Research and Writing'
course_df = dfm[filt_subject]
filt_student = course_df['value'].values.sum()
print(filt_student)
link = course_df.to_dict(orient = 'records')
SankeyWidget(links = link, margins=dict(top=10, bottom=0, left=250, right=60))

38.0


SankeyWidget(links=[{'source': 'Academic Research and Writing', 'target': 'A', 'value': 5.0}, {'source': 'Acad…